In [ ]:
import pandas as pd
import numpy as np
import wrds
import pathlib
import yfinance as yf
from pathlib import Path  

# Parent folder
pf = pathlib.Path().resolve() # Points to parent folder containing notebook and data, eg: 'C:/Users/Carla/Dropbox/Uni/10. Semester/Dynamic Programming/Term paper'

Process:
1. Download option chain data for from WRDS SECID by year 
2. Combine with data from other sources (yfinance, FRED)
3. Prepare data for estimation applying filters

# Download raw data

In [ ]:
# Connect to wrds
usr = '**'
db = wrds.Connection(wrds_username=usr)

# Creating password file - only for first time setup
db.create_pgpass_file() # PW = ******
#sorted(db.list_libraries()) # List of all available libraries
#db.list_tables(library="optionm") # List of all tables in optionm library
#db.describe_table(library="optionm", table="opprcd1996") # list of variables in table opprcd1996 - Note there is one table for each year e.g: oprcdXXXX

## Option data 

In [ ]:
# Specify which securityid (index) to get data for

secids = [108105, 102480, 102456, 102434, 143439, 101310, 12182]
names = ['SPX','NDX','DJX', 'RUT', 'TSLA', 'AMZN', 'GOOGL']

# Specify years
years = np.arange(1996, 2022, 1)

for i, secid in enumerate(secids):
    # Set up empty table to gather data for each year
    df = pd.DataFrame()

    # Get data for each year and append:
    for year in years:
        # Set up SQL expression to pass on
        sql = f"select secid, date, exdate, cp_flag, strike_price, best_bid, best_offer, volume, open_interest, impl_volatility, delta from optionm.opprcd{year} where secid = '{secid}'"

        # Get data 
        data_1year = db.raw_sql(sql)

        # Save yearly file (just in case)
        #filepath = Path(f'{pf}/data/input/allopt_{year}_extract.csv') # Set name 
        #data_1year.to_csv(filepath, index=False)

        # Append to main data
        df = pd.concat([df, data_1year])

    # Close the connection
    db.close()

    # Save data file
    filepath = Path(f'{pf}/data/input/rawdata_01jan1996to31dec2021_extract_{names[i]}.csv') # Set name 
    df.to_csv(filepath, index=False)

## Treasury rate data

Downloaded from FRED

4 week (1 month) - https://fred.stlouisfed.org/series/DTB4WK

3 months - https://fred.stlouisfed.org/series/DTB3

Before 2001-07-31: uses 3mo

On and after 2001-07-31: uses 1mo

In [ ]:
df_trate_1mo3mo = pd.read_excel(f'{pf}/data/input/trate_1mo3mo.xlsx', index_col=0)

## S&P 500 dividend yield

In [ ]:
# read data
df_snp_dy =  pd.read_excel(f'{pf}/data/input/div_shiller.xls')  # from # http://www.econ.yale.edu/~shiller/data.htm   

# resample with forward fill to put set the value of the month for each day 
df_snp_dy['date'] = pd.to_datetime(df_snp_dy.date, format='%d/%m/%Y')
df_snp_dy = df_snp_dy.set_index('date').resample('D').ffill()

## S&P 500 daily price data

In [ ]:
# Specify start and end date
date_start = '1995-01-01'
date_end = '2022-01-01'

# Get data with YFinance
df = yf.download('^GSPC', start=date_start, end=date_end)

# Rename columns 
df['snp'] = df['Adj Close']
df.index.names = ['date']

# Calculate 21 day moving average
df['snp21ma'] = df['snp'].rolling(21).mean()

# Drop unnecessary columns
df = df.drop(columns=['Open','High', 'Low', 'Close', 'Volume', 'Adj Close'])

# Save data file
filepath = Path(f'{pf}/data/input/SP500rawdata_1995-01-01to2021-12-31_extract.csv') # Set name 
df.to_csv(filepath, index=True)

df_snp = df

# Data processing

## MORE DATA: Indices and stocks

In [ ]:
names = ['SPX','NDX','DJX', 'RUT', 'TSLA', 'AMZN', 'GOOGL']
tickers = ['^GSPC','^NDX', '^DJI', '^RUT', 'TSLA', 'AMZN', 'GOOGL']

# Specify start and end date for yfinance
date_start = '1996-01-01'
date_end = '2022-01-01'

for i, (name, ticker) in enumerate(zip(names, tickers)):
    print(name, ticker)
    
    # Get yfinance data

    # Get data with YFinance
    df = yf.download(ticker, start=date_start, end=date_end)

    # Rename columns 
    df['snp'] = df['Adj Close']
    df.index.names = ['date']
    

    # Calculate 21 day moving average
    df['snp21ma'] = df['snp'].rolling(21).mean()

    # Drop unnecessary columns
    df = df.drop(columns=['Open','High', 'Low', 'Close', 'Volume', 'Adj Close'])

    # Save data file
    filepath = Path(f'{pf}/data/input/stockprices_{name}.csv') # Set name 
    df.to_csv(filepath, index=True)

    # To use in formatting
    df_snp = df
    df_snp = df_snp.reset_index()
    
    ### Option data
    
    # Read downloaded data
    df = pd.read_csv (f'{pf}/data/input/rawdata_01jan1996to31dec2021_extract_{name}.csv') # Data
    
    ### Format date columns
    df['date'] =  pd.to_datetime(df['date'], format='%Y-%m-%d')
    df['exdate'] =  pd.to_datetime(df['exdate'], format='%Y-%m-%d')
    df_snp['date'] =  pd.to_datetime(df_snp['date'], format='%Y-%m-%d')

    ### Merge with snp and treasury data
    df = pd.merge(left=df, right=df_snp, left_on="date", right_on="date", how="left")
    df = pd.merge(left=df, right=df_snp_dy, left_on="date", right_on="date", how="left")
    df = pd.merge(left=df, right=df_trate_1mo3mo, left_on="date", right_on="date", how="left")


    ### Calculated variables
    # Calculate tau (days to expiration)
    df['tau']=df['exdate']-df['date'] # shows in X days
    df['tau'] = (df['tau'] / np.timedelta64(1,'D')).astype(int) # as int

    # Calculate price (avg of bid/ask)
    df['price'] = (df['best_bid']+df['best_offer'])/2

    # Calculate strike (divide by 1000)
    df['strike'] = df['strike_price']/1000
    
    if name == 'DJX':
        df['strike']=df['strike_price']/10

    if name == 'AMZN': # Adjust for stock split
        df['snp']=df['snp']*20

    if name == 'GOOGL': # Adjust for stock split
        df['snp']=df['snp']*20

    if name == 'TSLA': # Adjust for stock split
        df['snp']=df['snp']*3
        
    # Moving average dividend yield
    df['dy_ma'] = df['dividend'] / df['snp21ma']

    # Moneyness variable
    df['money'] = np.log(df['strike']*np.exp(-(df['tr']-df['dy_ma'])*df['tau']/252)/df['snp'])

    ### Filters
    # Filter volume > 0
    df = df[df.volume > 0]

    # Filter bid/ask > 0.05
    df = df[(df['best_bid'] >=0.05) | (df['best_offer'] >=0.05)]
    df['spread']=df['best_offer']-df['best_bid']
    df = df[df.spread>=0]

    # Filter 8 < tau < 365
    #df = df[df.tau >= 9]
    df = df[df.tau <= 365]

    # Filter impl. vol > 0
    df = df.dropna(subset=['impl_volatility'])
    df = df[df.impl_volatility > 0]

    # Filter for non standard expiry dates 
    df['exdate_day'] = df.exdate.dt.day
    df['exdate_weekday'] = df.exdate.dt.dayofweek
    df = df[df['exdate_day'] < 27]
    df = df[df['exdate_weekday'] >= 4]

    # Filter for maturity groups
    df['tau_years'] = df['tau']/365 # tau in years
    #df['maturity_group'] = np.where(df['tau_years']<0.25, "low", np.where(df['tau_years']>0.5, "high", "med")) # group tau_years by, 0-0.25 = low, 0.25-0.50 = med, tau_years > 0.5 = high
    df['maturity_group'] = np.where(df['tau_years']<8/365, "vlow",  np.where((df['tau_years']<0.25)&(df['tau_years']<0.25), "low", np.where(df['tau_years']>0.5, "high", "med"))) #

    # create new table: count n by date, cp_flag, tau
    df_tcount = df.groupby(['date', 'cp_flag', 'tau']).size() #.groupby(level=1).max()
    df_tcount = df_tcount.to_frame(name='n')
    df_tcount = df_tcount.reset_index()

    # merge count to main table
    df = pd.merge(left=df, right=df_tcount, left_on=['date','cp_flag', 'tau'], right_on=['date','cp_flag', 'tau'], how='left')

    # keep only groups with n > 3
    df = df[df['n'] > 3]

    # Sort data
    df = df.sort_values(by=['date','cp_flag','exdate', 'tau', 'strike'])

    # Set index to date
    df = df.set_index('date')
    
    # Save processed data file
    filepath = Path(f'{pf}/data/output/allopt_01jan1996to31dec2021_extract_{name}.csv') # Set name 
    df.to_csv(filepath, index=True)
    
    print(name, ' - done', len(df))

In [ ]:
for name in names:
    # Save processed data file by year
    # full period
    start_dates = ['1996-01-01','1997-01-01','1998-01-01','1999-01-01','2000-01-01','2001-01-01','2002-01-01','2003-01-01','2004-01-01','2005-01-01','2006-01-01','2007-01-01','2008-01-01','2009-01-01','2010-01-01','2011-01-01','2012-01-01', '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01', '2017-01-01',
              '2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01']
    end_dates = ['1996-12-31','1997-12-31','1998-12-31','1999-12-31','2000-12-31','2001-12-31','2002-12-31','2003-12-31','2004-12-31','2005-12-31','2006-12-31','2007-12-31','2008-12-31','2009-12-31','2010-12-31','2011-12-31','2012-12-31', '2013-12-31', '2014-12-31', '2015-12-31', '2016-12-31', '2017-12-31',
              '2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31']

    df_main = pd.read_csv (f'{pf}/data/output/allopt_01jan1996to31dec2021_extract_{name}.csv')  # full data set

    #Yearly datasets
    for start, end in zip(start_dates, end_dates):
        ## Filter on date
        df = df_main[df_main['date'].between(start, end)]

        filepath = Path(f'{pf}/data/output/allopt_01Jan{start[0:4]}to31Dec{end[0:4]}_extract_{name}.csv') # Set name 
        df.to_csv(filepath, index=True)
        print(name, len(df))